In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [4]:
import pandas as pd
import pickle
from sklearn.cluster import KMeans
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

#X_total = pickle.load(open('X', 'rb'))
Y_fare_total = pickle.load(open('Y_fare', 'rb'))
#Y_tip_total = pickle.load(open('Y_tip', 'rb'))
#Y_triptime = pickle.load(open('Y_triptime', 'rb'))
#Y_fare_total = np.log(Y_fare_total)

X_total = pickle.load(open('X_Kaggle', 'rb'))
Y_triptime = np.log(pickle.load(open('Y_Kaggle_triptime', 'rb')))

In [5]:
from sklearn.model_selection import train_test_split

Xtrain, Xval,Ytrain,Yval = train_test_split(X_total,Y_triptime,test_size=0.2,random_state=0)


In [6]:
columns = []
for col in X_total.columns:
  if pd.api.types.is_string_dtype(X_total[col]): #is_string_dtype is pandas function
    columns.append(tf.feature_column.categorical_column_with_hash_bucket(col, 
        hash_bucket_size= len(X_total[col].unique())))

  elif pd.api.types.is_numeric_dtype(X_total[col]): #is_numeric_dtype is pandas function
    columns.append(tf.feature_column.numeric_column(col))


In [7]:
def train_input_fn(features, labels, batch_size):
    """An input function for training"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((features.to_dict(), labels.tolist()))

    # Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)

    # Build the Iterator, and return the read end of the pipeline.
    return dataset.make_one_shot_iterator().get_next()

In [8]:
import logging
#logging.getLogger().setLevel(logging.INFO)

In [9]:


run_config = tf.estimator.RunConfig().replace(
      session_config=tf.ConfigProto(
          device_count={'GPU': 1},
          log_device_placement=False))



In [10]:

print Xtrain.shape

model = tf.estimator.DNNRegressor(hidden_units=[20, 20], 
                                  feature_columns=columns, 
                                  config = run_config, 
                                  optimizer=tf.train.ProximalAdagradOptimizer( learning_rate=0.1,l1_regularization_strength=0.001))
model.train(input_fn=tf.estimator.inputs.pandas_input_fn(Xtrain,Ytrain,shuffle=True, batch_size=10000,num_epochs=100 ),max_steps=2000)

train_input = tf.estimator.TrainSpec(input_fn=tf.estimator.inputs.pandas_input_fn(Xtrain,Ytrain,shuffle=True), max_steps=5000 )
eval_input = tf.estimator.EvalSpec(input_fn=tf.estimator.inputs.pandas_input_fn(Xval,Yval,shuffle=True))

#tf.estimator.train_and_evaluate(model,train_input,eval_input)

(1166915, 134)
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': device_count {
  key: "GPU"
  value: 1
}
, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fb1e82d4a90>, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/tmp/tmp1EQcVq', '_save_summary_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmp1EQcVq/model.ckpt.
INFO:tensorflow:loss = 435750.25, step = 1
INFO:tensorflow:global_step/sec: 1.23279
INFO:tensorflow:loss = 5320.627, step = 101 (81.118 sec)
INFO:tensorflow:global_step/sec: 1.2473
INFO:tensorflow:loss = 4688.3857, step = 201 (80.174 sec)
INFO:tensorflow:global_step/sec: 1.25311
INFO:tensorf

KeyboardInterrupt: 

In [12]:
# Creates a graph.
a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess.run(c))

[[22. 28.]
 [49. 64.]]
